# Load 2017 Philly Weather Data to SQLite

#### In this notebook we will be loading information from our cleaned weather csv file in order to create another table in our SQLite database.

In [1]:
import pandas as pd
from sqlalchemy import create_engine, inspect, func
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, DateTime

In [2]:
# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()

In [3]:
# Create Database Connection

# If using this file to load data into Heroku Postgres database, change database_path to
# "postgres://..." URI address (amazonaws) and load one time.
database_path = '../flask_app/static/data/data_all.sqlite'
engine = create_engine(f'sqlite:///{database_path}')
conn = engine.connect()

In [4]:
# Confirm that our ppa table should already exist in the database since we created it first
engine.table_names()

['ppa']

In [5]:
# Code to get table names and data types...
# ----------------------------------------
# inspector = inspect(engine)
# columns = inspector.get_columns('ppa')
# for column in columns:
#     print(column["name"], column["type"])

In [6]:
# Create Weather Classes which will serve as the anchor points for our Weather Table
class Weather(Base):
    __tablename__ = 'weather'
    dt = Column(DateTime, primary_key=True)
    lat = Column(Float)
    lon = Column(Float)
    temp = Column(Float)
    feels_like = Column(Float)
    temp_min = Column(Float)
    temp_max = Column(Float)
    humidity = Column(Integer)
    wind_speed = Column(Float)
    wind_deg = Column(Integer)
    rain_1h = Column(Float)
    rain_3h = Column(Float)
    snow_1h = Column(Float)
    snow_3h = Column(Float)
    clouds_all = Column(Integer)
    weather_id = Column(String(20))
    weather_main = Column(String(30))
    weather_description = Column(String(300))
    weather_icon = Column(String(20))
    month = Column(Integer)
    day = Column(Integer)
    hour = Column(Integer)
    ymdh = Column(String(100))

In [7]:
# Create (if not already in existence) the table associated with our Weather classes.
Base.metadata.create_all(engine)

# Use this to clear out the db (if applicable)
# Base.metadata.drop_all(engine)

In [8]:
# Check for tables again
engine.table_names()

['ppa', 'weather']

In [9]:
# Read df
weather_df = pd.read_csv('../resources/cleaned_data/cleaned_philly_weather_2017.csv')
weather_df.head(2)

,dt,lat,lon,temp,feels_like,temp_min,temp_max,humidity,wind_speed,wind_deg,...,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon,month,day,hour,ymdh
0,2017-01-01 00:00:00,39.952584,-75.165222,42.51,27.28,39.2,44.83,42,18.34,200,...,0.0,90,804,Clouds,overcast clouds,04n,1,1,0,2017-01-01 00
1,2017-01-01 01:00:00,39.952584,-75.165222,42.73,30.25,39.2,45.94,46,13.87,210,...,0.0,75,803,Clouds,broken clouds,04n,1,1,1,2017-01-01 01


In [10]:
# Use pandas to load csv converted df into our sqlite database
weather_df.to_sql(name='weather', con=engine, if_exists='append', index=False)

In [11]:
# Check the data is successfully loaded in SQLite by performing a "select all" query
check_weather_info = pd.read_sql_query('select * from weather', con=engine)
check_weather_info.head()

,dt,lat,lon,temp,feels_like,temp_min,temp_max,humidity,wind_speed,wind_deg,...,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon,month,day,hour,ymdh
0,2017-01-01 00:00:00,39.952584,-75.165222,42.51,27.28,39.2,44.83,42,18.34,200,...,0.0,90,804,Clouds,overcast clouds,04n,1,1,0,2017-01-01 00
1,2017-01-01 01:00:00,39.952584,-75.165222,42.73,30.25,39.2,45.94,46,13.87,210,...,0.0,75,803,Clouds,broken clouds,04n,1,1,1,2017-01-01 01
2,2017-01-01 02:00:00,39.952584,-75.165222,43.79,30.16,41.0,46.70,46,16.11,210,...,0.0,90,804,Clouds,overcast clouds,04n,1,1,2,2017-01-01 02
3,2017-01-01 03:00:00,39.952584,-75.165222,44.01,30.34,41.0,46.58,45,16.11,220,...,0.0,90,804,Clouds,overcast clouds,04n,1,1,3,2017-01-01 03
4,2017-01-01 04:00:00,39.952584,-75.165222,45.27,30.87,42.8,46.40,41,17.22,220,...,0.0,90,804,Clouds,overcast clouds,04n,1,1,4,2017-01-01 04


In [12]:
# Check row count again to verify everything is loaded...
print(len(check_weather_info))

8760
